**IMPORTING PACKAGES**

In [69]:
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import geopandas as gpd
import utm
from shapely.geometry import Point, MultiPolygon
from shapely.geometry import Point, Polygon
from shapely.geometry import Point, MultiLineString
from shapely import wkt
import seaborn as sns
#!pip install --upgrade numpy==1.21
from haversine import haversine, Unit
import requests

## Esta librería no te servirá en este análisis, pero
## es la librería básica para scrapear páginas más sencillas

from bs4 import BeautifulSoup as bs
import undetected_chromedriver as uc
import pandas as pd
from io import StringIO


**BUILDING DATASET**

In [70]:
url_idealista_collected_data_1 = "https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/raw/fb65bda916b55a3f5199eba01688407dc47a1616/Data/Creation_Variables/Idealista_Collected_Data_1.csv"
url_idealista_collected_data_2 = "https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/raw/fb65bda916b55a3f5199eba01688407dc47a1616/Data/Creation_Variables/Idealista_Collected_Data_2.csv"
response = requests.get(url_idealista_collected_data_1)
content = response.content.decode("utf-8")
df1 = pd.read_csv(StringIO(content))
response = requests.get(url_idealista_collected_data_2)
content = response.content.decode("utf-8")
df2 = pd.read_csv(StringIO(content))
data = pd.concat([df1,df2])

In [71]:
print(len(data))
data.drop_duplicates(['id'],inplace=True)
print(len(data))

16323
16301


In [72]:
data.set_index('id', inplace=True)

**EXPORT LATITUDE, LONGITUDE LATER FOR CONSTRUCTION DEMONSTRATIVE MAPS**

In [73]:
export_latitude_longitude = False
if export_latitude_longitude:
    data[['longitude','latitude']].to_excel('long_lat_properties.xlsx')

**PROPERTY TYPE**

In [74]:
data['Flat'] =  np.where(data['title'].str.contains('Flat|flat'), 1, 0)
data['Penthouse']=  np.where(data['title'].str.contains('Penthouse'), 1, 0)
data['Duplex'] =   np.where(data['title'].str.contains('Duplex'), 1, 0)
data['House'] = np.where(data['title'].str.contains('House|house'), 1, 0)
data['House'] = np.where(data['title'].str.contains('Penthouse'), 0, data['House'])
#data['House'] = np.where(data['title'].str.contains('house'), 1, )

In [75]:
data[data['House'] == 1]

,Unnamed: 0.1,Unnamed: 0,title,minor_title,price,m2,basic_features,building_features,amenities,energy_consumption,...,info_features,last_update,reference_listing,prof_advertiser,private_advertiser,page_link,Flat,Penthouse,Duplex,House
id,,,,,,,,,,,,,,,,,,,,,
94780085,60,60,Detached house for sale in La Dreta de l'Eixa...,"Eixample, Barcelona","2,290,000",480,"Detached house, 3 floors, 480 m² built, 14 be...",NaN,NaN,138 kWh/m² year,...,"480, m², 14, bed.",Listing updated on 23 March,EVI0640,BARNES Barcelona,NaN,https://www.idealista.com/en/inmueble/94780085/,0,0,0,1
97338520,159,159,Terraced house for sale in La Dreta de l'Eixa...,"Eixample, Barcelona","1,350,000",209,"Terraced house, 209 m² built, 155 m² floor ar...",NaN,Air conditioning,93 kWh/m² year,...,"209, m², 3, bed.",Listing updated on 23 March,BCN06718,Bcn Advisors,NaN,https://www.idealista.com/en/inmueble/97338520/,0,0,0,1
96906053,308,308,Terraced house for sale in La Dreta de l'Eixa...,"Eixample, Barcelona","1,350,000",187,"Terraced house, 5 floors, 187 m² built, 3 bed...",NaN,Air conditioning,93 kWh/m² year,...,"187, m², 3, bed.",Listing updated on 1 March,W-02O2LQ,Barcelona MMC - Eixample - S2,NaN,https://www.idealista.com/en/inmueble/96906053/,0,0,0,1
98481442,312,312,Terraced house for sale in La Dreta de l'Eixa...,"Eixample, Barcelona","1,350,000",209,"Terraced house, 209 m² built, 138 m² floor ar...",NaN,Air conditioning,93 kWh/m² year,...,"209, m², 3, bed., Parking included",Listing updated on 3 March,VB2208002,aProperties Real Estate Barcelona,NaN,https://www.idealista.com/en/inmueble/98481442/,0,0,0,1
97746291,431,431,Semi-detached house for sale in Passatge de N...,"La Sagrada Família, Barcelona","260,000",85,"Semi-detached house, 2 floors, 85 m² built, 4...",NaN,Garden,NaN,...,"85, m², 4, bed.",Listing updated on 18 October,97746291,Insma Properties,NaN,https://www.idealista.com/en/inmueble/97746291/,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100965594,111,111,Terraced house for sale in Grau,"Sant Andreu, Barcelona","440,000",150,"Terraced house, 3 floors, 150 m² built, 142 m...",NaN,Air conditioning,121 kWh/m² year,...,"150, m², 3, bed., Parking included",Listing updated on 9 April,055-7680-0,Fincas Cumbre,NaN,https://www.idealista.com/en/inmueble/100965594/,0,0,0,1
101031290,183,183,Detached house for sale in calle de Llinars d...,"El Bon Pastor, Barcelona","88,000",81,"Detached house, 81 m² built, 3 bedrooms, 1 ba...",NaN,NaN,NaN,...,"81, m², 3, bed.",Listing updated on 5 April,1174,NaN,NaN,https://www.idealista.com/en/inmueble/101031290/,0,0,0,1
101078244,242,242,Terraced house for sale in calle de Recesvint,"Sant Andreu, Barcelona","515,000",145,"Terraced house, 2 floors, 145 m² built, 95 m²...",NaN,Air conditioning,240 kWh/m² year,...,"145, m², 4, bed.",Listing updated on 18 April,V0116-B,Eurofincas International Real Estate,NaN,https://www.idealista.com/en/inmueble/101078244/,0,0,0,1


In [76]:
data['check'] = data['Flat'] + data['Duplex'] +data['Penthouse']+ data['House'] 
#data = data[data['check'] == 1]
data.check.unique()

array([1, 0])

In [77]:
#data[data['check']==0].to_excel("check.xlsx")

**BUILDING FEATURES**

In [78]:
data.columns[:40]

Index(['Unnamed: 0.1', 'Unnamed: 0', 'title', 'minor_title', 'price', 'm2',
       'basic_features', 'building_features', 'amenities',
       'energy_consumption', 'energy_emmision', 'energy_label_consumption',
       'energy_label_emmision', 'street', 'subdistrict', 'district',
       'province', 'region_province', 'price_per_m2', 'description',
       'latitude', 'longitude', 'n_photos', 'info_features', 'last_update',
       'reference_listing', 'prof_advertiser', 'private_advertiser',
       'page_link', 'Flat', 'Penthouse', 'Duplex', 'House', 'check'],
      dtype='object')

In [79]:
unique_building_feat = set(list(data.building_features.unique()))
print(unique_building_feat)
data['building_features'] = data['building_features'].fillna('No')
data['Elevator'] = np.where(data['building_features'].str.contains('With lift'), 1, 0)
data['No Elevator'] = np.where(data['building_features'].str.contains('No lift'), 1, 0)
data['Basement'] = np.where(data['building_features'].str.contains('Basement'), 1, 0)
data['Ground floor'] = np.where(data['building_features'].str.contains('Ground floor'), 1, 0)
data['1th Floor'] = np.where(data['building_features'].str.contains('1st'), 1, 0)
data['2th Floor'] = np.where(data['building_features'].str.contains('2nd'), 1, 0)
data['3th Floor'] = np.where(data['building_features'].str.contains('3rd'), 1, 0)
data['Mezzanine'] = np.where(data['building_features'].str.contains('Mezzanine'), 1, 0)
data['Semi-basement'] = np.where(data['building_features'].str.contains('Semi-basement'), 1, 0)
for i in range(4,40):
    data[str(i)+'th Floor'] = np.where(data['building_features'].str.contains(str(i)+'th'), 1, 0)
    if i < 10:
         for t in range(10,40):
            #print(data.loc[data['building_features'].str.contains(str(t)+'th')==True])
            col =  str(i)+'th Floor'
            data.loc[data['building_features'].str.contains(str(t)+'th')==True, col] = 0 
        
    #if i > 10:
     #   for t in range(1,5):
      #      data.loc[data['building_features'].str.contains(str(t)+str(i)+'th') == True, str(i)+'th'] = 0        
data['exterior'] = np.where(data['building_features'].str.contains('exterior'), 1, 0)
data['interior'] = np.where(data['building_features'].str.contains('interior'), 1, 0)
data.rename(columns={'1th Floor': '1st Floor', '2th Floor': '2nd Floor'}, inplace=True)

{nan, '3rd floor, No lift', 'Floor -1, No lift', '18th floor exterior, With lift', 'Floor -1 exterior, With lift', '1st floor interior, No lift', '4th floor interior, With lift', 'Mezzanine interior, No lift', '3rd floor interior, No lift', '16th floor exterior, With lift', '10th floor exterior, No lift', '9th floor exterior, No lift', '4th floor, With lift', '4th floor exterior, With lift', '8th floor exterior, With lift', '14th floor exterior, With lift', '4th floor exterior, No lift', '7th floor, No lift', '5th floor, No lift', 'Ground floor exterior', '7th floor exterior, No lift', '4th floor interior', 'No lift', '9th floor, With lift', 'Basement exterior, With lift', '6th floor, With lift', 'Semi-basement, No lift', '2nd floor', '8th floor interior, With lift', 'Ground floor interior, With lift', '3rd floor exterior, No lift', '12th floor exterior, No lift', '1st floor interior, With lift', '6th floor exterior, With lift', '6th floor interior, With lift', 'Ground floor exterior, 

In [80]:
data[data["Semi-basement"] == 1]

,Unnamed: 0.1,Unnamed: 0,title,minor_title,price,m2,basic_features,building_features,amenities,energy_consumption,...,32th Floor,33th Floor,34th Floor,35th Floor,36th Floor,37th Floor,38th Floor,39th Floor,exterior,interior
id,,,,,,,,,,,,,,,,,,,,,
92047764,391,391,Flat / apartment for sale in El Fort Pienc,"Eixample, Barcelona","105,000",36,"36 m² built, 33 m² floor area, 1 bedroom, 1 ba...","Semi-basement exterior, With lift",NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
100926758,196,196,"Studio flat for sale in calle d'en Carabassa, 17","El Gòtic, Barcelona","155,000",60,"60 m² built, 55 m² floor area, No bedroom, 1 b...","Semi-basement exterior, With lift",NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
25992951,12,12,"Studio flat for sale in calle de Cantabria, 22","La Verneda i la Pau, Barcelona","235,000",130,"130 m² built, 120 m² floor area, No bedroom, 1...","Semi-basement interior, With lift","Air conditioning, Green areas",NaN,...,0,0,0,0,0,0,0,0,0,1
100890028,37,37,Flat / apartment for sale in calle de Cantabria,"La Verneda i la Pau, Barcelona","235,000",130,"130 m² built, 120 m² floor area, 1 bedroom, 1 ...","Semi-basement interior, With lift",Air conditioning,NaN,...,0,0,0,0,0,0,0,0,0,1
94362647,451,451,Flat / apartment for sale in El Guinardó,"Horta Guinardó, Barcelona","130,000",52,"52 m² built, 38 m² floor area, 2 bedrooms, 1 b...","Semi-basement interior, With lift",NaN,228 kWh/m² year,...,0,0,0,0,0,0,0,0,0,1
97154390,301,301,Flat / apartment for sale in calle de la Conca...,"El Carmel, Barcelona","115,000",71,"71 m² built, 50 m² floor area, 3 bedrooms, 1 b...","Semi-basement, No lift",NaN,208 kWh/m² year,...,0,0,0,0,0,0,0,0,0,0
98727650,331,331,Flat / apartment for sale in calle de la Conca...,"El Carmel, Barcelona","115,000",71,"71 m² built, 70 m² floor area, 3 bedrooms, 1 b...","Semi-basement exterior, No lift",NaN,208 kWh/m² year,...,0,0,0,0,0,0,0,0,1,0
97444855,382,382,Flat / apartment for sale in calle Gran Vista,"El Carmel, Barcelona","62,000",35,"35 m² built, 1 bedroom, 1 bathroom, Second han...","Semi-basement interior, No lift",NaN,401 kWh/m² year,...,0,0,0,0,0,0,0,0,0,1
98758589,452,452,Flat / apartment for sale in calle Doctor Leta...,"Horta, Barcelona","165,000",40,"40 m² built, 2 bedrooms, 1 bathroom, Second ha...","Semi-basement interior, No lift",NaN,NaN,...,0,0,0,0,0,0,0,0,0,1


In [81]:
#print check how many column of the building features contain no information
data[data.building_features.isna()]

,Unnamed: 0.1,Unnamed: 0,title,minor_title,price,m2,basic_features,building_features,amenities,energy_consumption,...,32th Floor,33th Floor,34th Floor,35th Floor,36th Floor,37th Floor,38th Floor,39th Floor,exterior,interior
id,,,,,,,,,,,,,,,,,,,,,


In [82]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'title', 'minor_title', 'price', 'm2',
       'basic_features', 'building_features', 'amenities',
       'energy_consumption', 'energy_emmision', 'energy_label_consumption',
       'energy_label_emmision', 'street', 'subdistrict', 'district',
       'province', 'region_province', 'price_per_m2', 'description',
       'latitude', 'longitude', 'n_photos', 'info_features', 'last_update',
       'reference_listing', 'prof_advertiser', 'private_advertiser',
       'page_link', 'Flat', 'Penthouse', 'Duplex', 'House', 'check',
       'Elevator', 'No Elevator', 'Basement', 'Ground floor', '1st Floor',
       '2nd Floor', '3th Floor', 'Mezzanine', 'Semi-basement', '4th Floor',
       '5th Floor', '6th Floor', '7th Floor', '8th Floor', '9th Floor',
       '10th Floor', '11th Floor', '12th Floor', '13th Floor', '14th Floor',
       '15th Floor', '16th Floor', '17th Floor', '18th Floor', '19th Floor',
       '20th Floor', '21th Floor', '22th Floor', '23th Floor

**AMENITIES**

In [83]:
unique_amenities_feat = set(list(data.amenities.unique()))
print(unique_amenities_feat)
data['Air conditioning'] = np.where(data['amenities'].str.contains('Air'), 1, 0)
data['Swimming pool'] = np.where(data['amenities'].str.contains('Swimming'), 1, 0)
data['Green Area'] = np.where(data['amenities'].str.contains('Green'), 1, 0)
data['Garden'] =  np.where(data['amenities'].str.contains('Garden'), 1, 0)

{nan, 'Air conditioning', 'Air conditioning, Swimming pool', 'Air conditioning, Swimming pool, Green areas', 'Green areas', 'Air conditioning, Garden', 'Swimming pool, Garden', 'Air conditioning, Green areas', 'Swimming pool', 'Garden', 'Swimming pool, Green areas', 'Air conditioning, Swimming pool, Garden'}


**BASIC FEATURES**

In [84]:
unique_basic_feat = str(set(list(data.basic_features.unique())))
unique_basic_feat = ','.join([unique_basic_feat]).split(',')
partial_words = ['built','condition','floor','bedroom','bathroom','Built','Balcony','Terrace','Orientation','heating',
               'renovating','Parking', 'West','South','Fitted wardrobes','reduced mobility','Storeroom', 'East'
                ,'parking', '000','New housing', 'Detached', 'Semi-detached']
unique_basic_feat_filter = [element for element in unique_basic_feat if not any(partial_word in element for partial_word in partial_words)]
unique_basic_feat_filter

[' Land plot of 400 m²',
 ' Land plot of 607 m²',
 ' Land plot of 140 m²',
 " ' House",
 ' Land plot of 220 m²',
 ' Land plot of 287 m²',
 '500',
 ' Land plot of 219 m²',
 ' Land plot of 600 m²',
 ' Land plot of 433 m²',
 ' Land plot of 117 m²',
 ' Land plot of 150 m²',
 ' Land plot of 1',
 '069 m²',
 ' Land plot of 760 m²',
 " ' House",
 ' Land plot of 1',
 '070 m²',
 ' Land plot of 450 m²',
 " ' House",
 ' Land plot of 1',
 '073 m²',
 '500',
 ' Land plot of 70 m²',
 " ' House",
 ' Land plot of 58 m²',
 ' Land plot of 2',
 '139 m²',
 ' Land plot of 190 m²',
 ' Land plot of 198 m²',
 ' Land plot of 938 m²',
 ' Land plot of 387 m²',
 " ' House",
 ' 1',
 ' Land plot of 1',
 '696 m²',
 " ' House",
 ' Land plot of 90 m²',
 " ' House",
 ' Land plot of 312 m²',
 ' 1',
 ' Land plot of 1',
 '930 m²',
 ' Land plot of 241 m²',
 ' Land plot of 1',
 " ' House",
 ' Land plot of 1',
 '065 m²',
 ' Land plot of 796 m²',
 " ' House",
 ' Land plot of 200 m²',
 ' Land plot of 757 m²',
 " ' House",
 ' Lan

**INFO FEATURES**

In [85]:
unique_info_feat = str(set(list(data.info_features.unique())))
unique_info_feat = ','.join([unique_info_feat]).split(',')
partial_words = ['built','condition','floor','bedroom','bathroom','Built','Balcony','Terrace','Orientation','heating',
               'renovating','Parking', 'West','South','Fitted wardrobes','reduced mobility','Storeroom', 'East'
                ,'parking', '000','New housing', 'Detached', 'Semi-detached', 'exterior', 'm²', '1', '2', '3', '4', '5'
                ,'6', '7','8','9', 'bed', 'lift']
unique_info_feat_filter = [element for element in unique_info_feat if not any(partial_word in element for partial_word in partial_words)]
set(unique_info_feat_filter)

{" 'Finished new home",
 " 'New home",
 ' Basement',
 " Garage optional'",
 ' Mezzanine',
 " Mezzanine'",
 ' Semi-basement',
 " interior'"}

In [86]:
for index, row in data.iterrows():
    a = row['basic_features']

In [87]:
a

'90 m² built, 80 m² floor area, 3 bedrooms, 1 bathroom, Additional parking space for € 25,000, Second hand/good condition, Fitted wardrobes, Orientation South, Built in 1960,  Individual heating: Natural gas, Only outdoor access is adapted for people with reduced mobility'

In [88]:
list_built_m2 = []
list_floor_m2 = []
list_n_bedroom = []
list_n_bathroom = []
list_condition = []
list_build_year = []
list_terrace = []
list_heating = []
list_balcony = []
list_parking = []
list_detached = []
list_disabled = []

for index, row in data[['basic_features','info_features']].iterrows():
    
    "Basic Features"
    basic_feat = ','.join([row['basic_features']]).split(',')
    built_m2 = '.'.join([element for element in basic_feat if 'built' in element])
    floor_m2 = '.'.join([element for element in basic_feat if 'floor' in element])
    n_bedroom = '.'.join([element for element in basic_feat if 'bedroom' in element])
    n_bathroom = '.'.join([element for element in basic_feat if 'bathroom' in element])
    condition = [element for element in basic_feat if 'condition' in element]
    condition += [element for element in basic_feat if 'renovating' in element]
    condition += [element for element in basic_feat if 'New housing' in element]
    condition = '.'.join([element for element in condition if not 'Air' in element])
    build_year ='.'.join([element for element in basic_feat if 'Built in' in element]) 
    terrace = '.'.join([element for element in basic_feat if 'Terrace' in element])
    heating = '.'.join([element for element in basic_feat if 'heating' in element]) 
    individual_heating = str([element for element in basic_feat if 'Individual heating' in element]) 
    balcony =  '.'.join([element for element in basic_feat if 'Balcony' in element]) 
    disabled = '.'.join([element for element in basic_feat if 'reduced mobility' in element])
    parking = [element for element in basic_feat if 'Parking' in element]
    parking += [element for element in basic_feat if 'parking' in element]
    #parking = '.'.join([element for element in parking])
    detached = [element for element in basic_feat if 'Detached' in element]  
    detached += [element for element in basic_feat if 'Semi-detached' in element]
    detached = '.'.join([element for element in detached])

    "Info Features"
    info_feat = ','.join([row['info_features']]).split(',')
    if n_bedroom == "":
        n_bedroom = '.'.join([element for element in info_feat if 'bedroom' in element])  
    if n_bathroom == "":
        n_bathroom = '.'.join([element for element in info_feat if 'bathroom' in element])
    if build_year == "":
        build_year ='.'.join([element for element in info_feat if 'Built in' in element]) 
    if terrace == "":
        terrace += '.'.join([element for element in info_feat if 'Terrace' in element])
    if heating == "":
        heating += '.'.join([element for element in info_feat if 'heating' in element]) 
    if individual_heating == "":
        individual_heating = str([element for element in info_feat if 'Individual heating' in element]) 
    if balcony == "":
        balcony += '.'.join([element for element in info_feat if 'balcony' in element]) 
    if disabled == "":
        disabled += '.'.join([element for element in info_feat if 'reduced mobility' in element]) 
    if parking == []:
        parking += [element for element in info_feat if 'Parking' in element]
        parking += [element for element in info_feat if 'parking' in element]
    parking = '.'.join([element for element in parking])
    if detached == "":
        detached = [element for element in info_feat if 'Detached' in element]  
        detached += [element for element in info_feat if 'Semi-detached' in element]
        detached = '.'.join([element for element in detached])
        
        
    
#info features


    #List Appending
    list_built_m2.append(built_m2)
    list_floor_m2.append(floor_m2)
    list_n_bedroom.append(n_bedroom)
    list_n_bathroom.append(n_bathroom)
    list_condition.append(condition)
    list_build_year.append(build_year)
    list_terrace.append(terrace)
    list_heating.append(heating)
    list_balcony.append(balcony)
    list_parking.append(parking)
    list_detached.append(detached)
    list_disabled.append(disabled)

In [89]:
set(list_condition)

{' New housing development',
 ' Second hand/good condition',
 ' Second hand/needs renovating'}

**CHECK OCCURENCES ELEMENTS IN LIST**

In [90]:
def element_n_timesinlist(list_name):
    for el in set(list_name):
        if el == "":
            print(f'Empty is {list_name.count(el)}')
        else:
            print(f'Element: {el} is {list_name.count(el)}')
    print("")  
    return

*ROOMS*

In [91]:
element_n_timesinlist(list_n_bedroom)


Element:  14 bedrooms is 1
Element:  12 bedrooms is 3
Element:  8 bedrooms is 35
Element:  7 bedrooms is 95
Element:  4 bedrooms is 3099
Element:  55 bedrooms is 1
Element:  16 bedrooms is 1
Element:  No bedroom is 234
Element:  6 bedrooms is 301
Element:  2 bedrooms is 4155
Element:  9 or more bedrooms is 14
Element:  17 bedrooms is 2
Element:  50 bedrooms is 1
Element:  3 bedrooms is 6023
Element:  11 bedrooms is 9
Element:  5 bedrooms is 960
Element:  15 bedrooms is 3
Element:  10 bedrooms is 16
Element:  1 bedroom is 1348



*BATHROOMS*

In [92]:
element_n_timesinlist(list_n_bathroom)

Element:  No bathroom is 6
Element:  8 bathrooms is 23
Element:  4 bathrooms is 495
Element:  7 bathrooms is 47
Element:  1 bathroom is 7725
Element:  2 bathrooms is 6185
Element:  5 bathrooms is 185
Element:  10 bathrooms is 1
Element:  3 bathrooms is 1550
Element:  6 bathrooms is 80
Element:  9 bathrooms is 4



*CONDITION*

In [93]:
element_n_timesinlist(list_condition)

Element:  Second hand/good condition is 13378
Element:  New housing development is 654
Element:  Second hand/needs renovating is 2269



*TERRACE*

In [94]:
element_n_timesinlist(list_terrace)

Empty is 10566
Element:  Terrace is 5583
Element:  Terraced house. Terrace is 134
Element:  Terraced house is 18



*HEATING*

In [95]:
element_n_timesinlist(list_heating)

Empty is 5960
Element:   Individual heating: Propane/butane gas is 6
Element:   Central heating: Electric is 1
Element:   Individual heating: Gas is 4
Element:   No heating: Natural gas is 3
Element:   Central heating: Fuel oil is 24
Element:   Individual heating: Natural gas is 3473
Element:   Central heating: Gas is 331
Element:   Central heating is 558
Element:   Individual heating: Electric is 705
Element:   Central heating: Natural gas is 1
Element:   No heating: Electric is 2
Element:   Individual heating is 2012
Element:   Central heating: Air conditioning/Heat pump is 1
Element:   No heating is 2302
Element:   Individual heating: Air conditioning/Heat pump is 918



*BALCONY*

In [96]:
element_n_timesinlist(list_balcony)

Empty is 9744
Element:  Balcony is 6557



*PARKING*

In [97]:
element_n_timesinlist(list_parking)

Empty is 13906
Element:  Additional parking space for € 48 is 3
Element:  Additional parking space for € 30 is 73
Element:  Additional parking space for € 28 is 9
Element:  Additional parking space for € 90 is 1
Element:  Additional parking space for € 150 is 4
Element:  Additional parking space for € 14 is 3
Element:  Additional parking space for € 39 is 2
Element:  Additional parking space for € 12 is 4
Element:  Additional parking space for € 24 is 15
Element:  Additional parking space for € 5 is 1
Element:  Additional parking space for € 65 is 1
Element:  Additional parking space for € 20 is 48
Element:  Additional parking space for € 37 is 3
Element:  Additional parking space for € 16 is 5
Element:  Additional parking space for € 55 is 3
Element:  Additional parking space for € 160 is 1
Element:  Additional parking space for € 34 is 14
Element:  Additional parking space for € 25 is 89
Element:  Additional parking space for € 35 is 21
Element:  Additional parking space for € 1 is 5

*DETACHED*

In [98]:
element_n_timesinlist(list_detached)

Empty is 15979
Element:  Detached house is 254
Element:  Semi-detached house is 68



*DISABLED*

In [99]:
element_n_timesinlist(list_disabled)

Empty is 14248
Element:  Only the interior of the property is adapted for people with reduced mobility. is 174
Element:  Access and property adapted for people with reduced mobility is 785
Element:  Only outdoor access is adapted for people with reduced mobility is 1094



**USE LISTS FOR COLUMNS IN DATAFRAME**

In [100]:
data['built_m2'] = [int(built.split()[0]) for built in list_built_m2]
data['floor_m2'] = [float(area.split()[0]) if area else np.nan for area in list_floor_m2]
#bedrooms
data['bedrooms'] = [(bedroom.split()[0]) for bedroom in list_n_bedroom]
data['bedrooms'] = data['bedrooms'].replace('', np.NaN)
data['bedrooms'] = data['bedrooms'].replace('No','0')
data['bedrooms'] = data['bedrooms'].astype(int)
#bathroom
data['bathrooms'] = [(bathroom.split()[0]) for bathroom in list_n_bathroom]
data['bathrooms'] = data['bathrooms'].replace('', np.NaN)
data['bathrooms'] = data['bathrooms'].replace('No','0')
data['bathrooms'] = data['bathrooms'].astype(int)
#conditions
print(set(list_condition))
list_conditions = [np.NaN if x == '' else x for x in list_condition]
data['condition'] = list_condition
dummies_condition = pd.get_dummies(data['condition'])
data = pd.concat([data, dummies_condition],axis=1)

#build year
data['build_year'] = [int(area.split()[2]) if area else np.nan for area in list_build_year]
#terrace
print(set(list_terrace))
data['Terrace'] = [1 if value == ' Terrace' else 0 for value in list_terrace] 
#heating
print(set(list_heating))
list_heating = [np.NaN if x == '' else x for x in list_heating]
data['Heating_info'] = list_heating
data['Heating'] = [0 if value in [np.NaN,"  No heating"] else 1 for value in list_heating] 
data['Central_heating'] = [1 if value in ['  Central heating', '  Central heating: Fuel oil','  Central heating: Gas'] else 0 for value in list_heating] 
data['Individual_heating'] = [1 if value in ['  Individual heating: Natural gas', '  Individual heating: Air conditioning/Heat pump', '  Individual heating: Electric'] else 0 for value in list_heating]
#balcony
print(set(list_balcony))
data['Balcony'] = [1 if value in [" Balcony"] else 0 for value in list_balcony] 
#parking
print(set(list_parking))
data['Parking_included'] = [1 if value in [" Parking space included in the price"] else 0 for value in list_parking] 
data['Parking_payable'] =  [1 if 'Additional' in word else 0 for word in list_parking]
#new housing
#print(set(list_new_housing))
#data['new_housing_development'] = [1 if value in [' New housing development'] else 0 for value in list_new_housing] 

{' Second hand/good condition', ' New housing development', ' Second hand/needs renovating'}
{'', ' Terrace', ' Terraced house. Terrace', ' Terraced house'}
{'', '  Individual heating: Propane/butane gas', '  Central heating: Electric', '  Individual heating: Gas', '  No heating: Natural gas', '  Central heating: Fuel oil', '  Individual heating: Natural gas', '  Central heating: Gas', '  Central heating', '  Individual heating: Electric', '  Central heating: Natural gas', '  No heating: Electric', '  Individual heating', '  Central heating: Air conditioning/Heat pump', '  No heating', '  Individual heating: Air conditioning/Heat pump'}
{'', ' Balcony'}
{'', ' Additional parking space for € 48', ' Additional parking space for € 30', ' Additional parking space for € 28', ' Additional parking space for € 90', ' Additional parking space for € 150', ' Additional parking space for € 14', ' Additional parking space for € 39', ' Additional parking space for € 12', ' Additional parking space f

In [101]:
print(set(list_heating))

{nan, '  Individual heating: Propane/butane gas', '  Central heating: Electric', '  Individual heating: Gas', '  No heating: Natural gas', '  Central heating: Fuel oil', '  Individual heating: Natural gas', '  Central heating: Gas', '  Central heating', '  Individual heating: Electric', '  Central heating: Natural gas', '  No heating: Electric', '  Individual heating', '  Central heating: Air conditioning/Heat pump', '  No heating', '  Individual heating: Air conditioning/Heat pump'}


**ENERGY LABEL**

In [102]:
print(data['energy_label_emmision'].unique())
print(data['energy_label_consumption'].unique())
#labels
for label in ['A','B','C','D','E', 'F', 'G']:
    data['energy_label_emmision_'+(str(label))] = 0
    data.loc[data['energy_label_emmision'] == label, 'energy_label_emmision_'+(str(label))] = 1
    data['energy_label_consumption_'+(str(label))] = 0
    data.loc[data['energy_label_consumption'] == label.lower(), 'energy_label_consumption_'+(str(label))] = 1
data['energy_label_emission_NA'] = [1 if value in ['In process','Consumption: ','Not indicated', 'Exempt'] else 0 for value in data['energy_label_emmision']] 
data['energy_label_consumption_NA'] = [1 if value in ['In process','Consumption: ','Not indicated', 'Exempt'] else 0 for value in data['energy_label_consumption']] 
#Numerical Data
data['energy_emmision_number'] =  [(x.split()[0]) if isinstance(x, str) and 'kg CO2' in x else x for x in data['energy_emmision']]
data['energy_emmision_number'] = data['energy_emmision_number'].str.replace(',', '.').astype(float)

data['energy_consumption_number'] =  [(x.split()[0]) if isinstance(x, str) and 'kWh' in x else x for x in data['energy_consumption']]
data['energy_consumption_number'] = data['energy_consumption_number'].str.replace(',', '.').astype(float)


['G' 'Consumption: ' 'C' 'In process' 'Not indicated' 'E' 'B' 'F' 'D'
 'Exempt' 'A']
['g' 'd' 'c' 'In process' 'e' 'Not indicated' 'b' 'f' 'Exempt' 'a']


**INFO FEATURES**

In [103]:
unique_info_feat = str(set(list(data.info_features.unique())))
unique_info_feat = ','.join([unique_info_feat]).split(',')
partial_words = ['built','condition','floor','bedroom','bathroom','Built','Balcony','Terrace','Orientation','heating',
               'renovating','Parking', 'West','South','Fitted wardrobes','reduced mobility','Storeroom', 'East'
                ,'parking', '000','New housing', 'Detached', 'Semi-detached', 'exterior', 'm²', '1', '2', '3', '4', '5'
                ,'6', '7','8','9', 'bed', 'lift']
unique_info_feat_filter = [element for element in unique_info_feat if not any(partial_word in element for partial_word in partial_words)]
set(unique_info_feat_filter)

{" 'Finished new home",
 " 'New home",
 ' Basement',
 " Garage optional'",
 ' Mezzanine',
 " Mezzanine'",
 ' Semi-basement',
 " interior'"}

In [104]:
list_luxury = []
list_new_home = []
for row in data.info_features:
    list_row = ','.join([row]).split(',')
    luxury = '.'.join([element for element in list_row if 'Lux' in element])
    new_home = '.'.join([element for element in list_row if 'New home' in element])
    list_luxury.append(luxury)
    list_new_home.append(new_home)
print(set(list_luxury))
print(set(list_new_home))
data['luxury'] = [1 if value in [" Luxury"] else 0 for value in list_luxury] 
data['new_home'] =  [1 if value in ["New home"] else 0 for value in list_new_home] 

{''}
{'', 'New home'}


**CORRECT DATA FORMAT PRICE**

In [105]:
data['price'] = data['price'].str.strip()
data['price'] =   data['price'].str.replace(',', '').astype(int)


**CORRECT M2 FORMAT**

In [106]:
#data = data.drop(data[data['m2']== 'Finished new home'].index)
#data.m2 = data.m2.str.replace(',','')
#data.m2 = data.m2.astype(float)

**AGE**

In [107]:
#building year is set to age in-line with earlier research about barcelona (Marmolejo-Duarte et al. 2022, Buonanno et al. 2013)
data['age'] = 2023 - data.build_year

**CHECK**

In [108]:
data.columns[:40]

Index(['Unnamed: 0.1', 'Unnamed: 0', 'title', 'minor_title', 'price', 'm2',
       'basic_features', 'building_features', 'amenities',
       'energy_consumption', 'energy_emmision', 'energy_label_consumption',
       'energy_label_emmision', 'street', 'subdistrict', 'district',
       'province', 'region_province', 'price_per_m2', 'description',
       'latitude', 'longitude', 'n_photos', 'info_features', 'last_update',
       'reference_listing', 'prof_advertiser', 'private_advertiser',
       'page_link', 'Flat', 'Penthouse', 'Duplex', 'House', 'check',
       'Elevator', 'No Elevator', 'Basement', 'Ground floor', '1st Floor',
       '2nd Floor'],
      dtype='object')

In [109]:
check_var = []
for i in range(1,40):
    if i == 1:
        check_var.append('1st Floor')
    elif i == 2:
        check_var.append('2nd Floor')
    else:
        check_var.append(str(i)+"th Floor")
check_var.append('Basement')
check_var.append('Ground floor')
check_var.append('Mezzanine')
check_var.append('Semi-basement')
data['check'] = (data[check_var]).sum(axis=1)
data['check'].unique()

array([1, 0], dtype=int64)

In [110]:
check_var

['1st Floor',
 '2nd Floor',
 '3th Floor',
 '4th Floor',
 '5th Floor',
 '6th Floor',
 '7th Floor',
 '8th Floor',
 '9th Floor',
 '10th Floor',
 '11th Floor',
 '12th Floor',
 '13th Floor',
 '14th Floor',
 '15th Floor',
 '16th Floor',
 '17th Floor',
 '18th Floor',
 '19th Floor',
 '20th Floor',
 '21th Floor',
 '22th Floor',
 '23th Floor',
 '24th Floor',
 '25th Floor',
 '26th Floor',
 '27th Floor',
 '28th Floor',
 '29th Floor',
 '30th Floor',
 '31th Floor',
 '32th Floor',
 '33th Floor',
 '34th Floor',
 '35th Floor',
 '36th Floor',
 '37th Floor',
 '38th Floor',
 '39th Floor',
 'Basement',
 'Ground floor',
 'Mezzanine',
 'Semi-basement']

In [111]:
#data[data['check']==2].to_excel('check.xlsx')

**DUMMIES REGION**

In [112]:
print(f'Unique values for province {data.province.unique()}')
print(f'Unique values for region provinces {data.region_province.unique()}')
print(f'Unique values for districts {data.district.unique()}')
dummies_district_names = data.district.unique()

Unique values for province ['Barcelona']
Unique values for region provinces ['Barcelona region, Barcelona']
Unique values for districts ['District Eixample' 'District Ciutat Vella' 'District Sant Martí'
 'District Sants-Montjuïc' 'District Horta Guinardó' 'District Gràcia'
 'District Les Corts' 'District Nou Barris' 'District Sarrià-Sant Gervasi'
 'District Sant Andreu']


In [113]:
dummies_district = pd.get_dummies(data['district'])
data = pd.concat([data, dummies_district],axis=1)

# MODEFYING DATA

**FEATURE SELECTION**


In [114]:
for col in data.columns:
    print(col)

Unnamed: 0.1
Unnamed: 0
title
minor_title
price
m2
basic_features
building_features
amenities
energy_consumption
energy_emmision
energy_label_consumption
energy_label_emmision
street
subdistrict
district
province
region_province
price_per_m2
description
latitude
longitude
n_photos
info_features
last_update
reference_listing
prof_advertiser
private_advertiser
page_link
Flat
Penthouse
Duplex
House
check
Elevator
No Elevator
Basement
Ground floor
1st Floor
2nd Floor
3th Floor
Mezzanine
Semi-basement
4th Floor
5th Floor
6th Floor
7th Floor
8th Floor
9th Floor
10th Floor
11th Floor
12th Floor
13th Floor
14th Floor
15th Floor
16th Floor
17th Floor
18th Floor
19th Floor
20th Floor
21th Floor
22th Floor
23th Floor
24th Floor
25th Floor
26th Floor
27th Floor
28th Floor
29th Floor
30th Floor
31th Floor
32th Floor
33th Floor
34th Floor
35th Floor
36th Floor
37th Floor
38th Floor
39th Floor
exterior
interior
Air conditioning
Swimming pool
Green Area
Garden
built_m2
floor_m2
bedrooms
bathrooms
cond

In [115]:
raw_features= ['built_m2',"Flat",
    "Penthouse","Duplex","House",
           *check_var,
           'Elevator','Swimming pool', 'Green Area','Garden','Air conditioning',
           'bedrooms','bathrooms',
           #'exterior','interior',
           'age',
           'Terrace', 'Balcony',
           'Heating',
           'Parking_included', #'parking_payable',
           #'luxury', 'new_home',
        ' New housing development',
        ' Second hand/good condition',
       ' Second hand/needs renovating',
        'energy_label_emmision_A','energy_label_consumption_A',
       'energy_label_emmision_B', 'energy_label_consumption_B',
       'energy_label_emmision_C', 'energy_label_consumption_C',
       'energy_label_emmision_D', 'energy_label_consumption_D',
       'energy_label_emmision_E', 'energy_label_consumption_E',
        'energy_label_emmision_F', 'energy_label_consumption_F',
        'energy_label_emmision_G', 'energy_label_consumption_G',
       'energy_label_emission_NA', 'energy_label_consumption_NA',
       *dummies_district_names, 'subdistrict', 'price']
#features.append(dummies_district)
df_features = data[raw_features].copy(deep=True)
#target = ['price']
#df_target = data[target].copy(deep=True)

**RENAME COLUMNS**

In [116]:
df_features.rename(columns={'built_m2': 'Building surface m²', 'Parking_included': 'Parking included','energy_label_consumption_A':'Energy label consumption A'
                            ,'energy_label_consumption_B':'Energy label consumption B', 'energy_label_consumption_C':'Energy label consumption C'
                           ,'energy_label_consumption_D':'Energy label consumption D','energy_label_consumption_E':'Energy label consumption E',
                           'energy_label_consumption_F':'Energy label consumption F','energy_label_consumption_G': 'Energy label consumption G',
                            'energy_label_consumption_NA':'Energy label consumption NA',
                           ' New housing development' : 'New housing development',' Second hand/good condition':'Second hand/good condition',
                            ' Second hand/needs renovating':  'Second hand/needs renovating', 'age': 'Building age', "Ground floor": 'Ground Floor'}, inplace=True)

In [117]:
df_features['Ground Floor']

id
31702050     0
33148486     0
39158016     0
40351755     0
93970881     0
            ..
101135260    0
101135697    0
101135655    0
101134507    0
101137418    0
Name: Ground Floor, Length: 16301, dtype: int32

**TREATING MISSING VALUES**

**CHECKING FLOOR DATA**

In [118]:
check_var = []
for i in range(1,40):
    if i == 1:
        check_var.append('1st Floor')
    elif i == 2:
        check_var.append('2nd Floor')
    else:
        check_var.append(str(i)+"th Floor")
check_var.append('Basement')
check_var.append('Ground Floor')
check_var.append("Mezzanine")
check_var.append('Semi-basement')
df_features['check'] = (df_features[check_var]).sum(axis=1)
df_features['check'].unique()
df_features['check'] = (df_features[check_var]).sum(axis=1)
print(f'Unique values for the floor {df_features.check.unique()}') 
print(f'Number of properties with floor information is: {(df_features.check !=0).sum()} out of: {len(df_features)}')
data[df_features.check == 0].to_excel('removed_if_nanfloordropped.xlsx')

Unique values for the floor [1 0]
Number of properties with floor information is: 14388 out of: 16301


In [119]:
len(data[df_features.check == 2])

0

In [120]:
df_features[check_var].sum()

1st Floor        3301
2nd Floor        2500
3th Floor        1917
4th Floor        1539
5th Floor        1216
6th Floor         635
7th Floor         358
8th Floor         249
9th Floor         113
10th Floor        105
11th Floor         30
12th Floor         14
13th Floor         12
14th Floor         33
15th Floor          8
16th Floor         18
17th Floor          5
18th Floor          2
19th Floor          6
20th Floor          4
21th Floor          0
22th Floor          0
23th Floor          0
24th Floor          6
25th Floor          4
26th Floor          0
27th Floor          0
28th Floor          0
29th Floor          0
30th Floor          0
31th Floor          0
32th Floor          0
33th Floor          0
34th Floor          1
35th Floor          0
36th Floor          0
37th Floor          0
38th Floor          0
39th Floor          0
Basement           39
Ground Floor     1526
Mezzanine         717
Semi-basement      30
dtype: int64

In [121]:
higher_5_col = []
for i in range(6,40):
    higher_5_col.append(str(i)+'th Floor')
higher_5_col

['6th Floor',
 '7th Floor',
 '8th Floor',
 '9th Floor',
 '10th Floor',
 '11th Floor',
 '12th Floor',
 '13th Floor',
 '14th Floor',
 '15th Floor',
 '16th Floor',
 '17th Floor',
 '18th Floor',
 '19th Floor',
 '20th Floor',
 '21th Floor',
 '22th Floor',
 '23th Floor',
 '24th Floor',
 '25th Floor',
 '26th Floor',
 '27th Floor',
 '28th Floor',
 '29th Floor',
 '30th Floor',
 '31th Floor',
 '32th Floor',
 '33th Floor',
 '34th Floor',
 '35th Floor',
 '36th Floor',
 '37th Floor',
 '38th Floor',
 '39th Floor']

In [122]:
df_features['6th Floor or higher'] = (df_features[[*higher_5_col]]).sum(axis=1)

**CHECK DUMMIES**

In [123]:
check_dummies = df_features[df_features[[*dummies_district]].sum(axis=1)!=1]
check_dummies

,Building surface m²,Flat,Penthouse,Duplex,House,1st Floor,2nd Floor,3th Floor,4th Floor,5th Floor,...,District Horta Guinardó,District Gràcia,District Les Corts,District Nou Barris,District Sarrià-Sant Gervasi,District Sant Andreu,subdistrict,price,check,6th Floor or higher
id,,,,,,,,,,,,,,,,,,,,,


*comment* <br> <br>
Sum of Dummies Always Exact 1. Data is Correct

**CHECKING ENERGY LABEL**

In [124]:
df_features['check'] = (df_features[[ 'energy_label_emmision_A','Energy label consumption A',
       'energy_label_emmision_B', 'Energy label consumption B',
       'energy_label_emmision_C', 'Energy label consumption C',
       'energy_label_emmision_D', 'Energy label consumption D',
       'energy_label_emmision_E', 'Energy label consumption E',
        'energy_label_emmision_F', 'Energy label consumption F',
        'energy_label_emmision_G', 'Energy label consumption G',
       'energy_label_emission_NA', 'Energy label consumption NA',]]).sum(axis=1)
print(f'Unique values for the energy label {df_features.check.unique()}') 
print(f'Number of properties with no complete energy label is: {(df_features.check !=2).sum()} out of: {len(df_features)}')
#data[df_features.check != 2].to_excel('check.xlsx')
length = (df_features['Energy label consumption NA'] == 1).sum()
print(f'Number of no energy label for consumption is {length} out of: {len(df_features)}')
print(f'Number of no energy label for emmision is {(df_features.energy_label_emission_NA == 1).sum()} out of: {len(df_features)}')
both_mis = len(df_features[(df_features['energy_label_emission_NA']== 1) & (df_features['Energy label consumption NA'] == 1)])
print(f'Number of no energy label for both consumption and emmision is: {both_mis}')

Unique values for the energy label [2]
Number of properties with no complete energy label is: 0 out of: 16301
Number of no energy label for consumption is 3970 out of: 16301
Number of no energy label for emmision is 5652 out of: 16301
Number of no energy label for both consumption and emmision is: 3970


In [125]:
#data[df_features['energy_label_emission_NA']== 1].to_excel('check.xlsx')

In [126]:
#data[df_features['Building surface m²'].isna()].to_excel('check.xlsx')

**feedback**
- Missing Floor Data Droppen?
<br> <br>
- Missing Energy Label Data Read Papers

In [127]:
for column in df_features.columns:
    print(f'Missing Values for Column {column} : {df_features[column].isna().sum()} out {len(df_features)}, which is {df_features[column].isna().sum()/len(df_features)} percent')

Missing Values for Column Building surface m² : 0 out 16301, which is 0.0 percent
Missing Values for Column Flat : 0 out 16301, which is 0.0 percent
Missing Values for Column Penthouse : 0 out 16301, which is 0.0 percent
Missing Values for Column Duplex : 0 out 16301, which is 0.0 percent
Missing Values for Column House : 0 out 16301, which is 0.0 percent
Missing Values for Column 1st Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 2nd Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 3th Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 4th Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 5th Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 6th Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 7th Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 8th Floor : 0 out 16301, which is 0.0 percent
Missing Values for Column 9th Floor : 0 out 16301, which is 0.0 pe

**feedback** <br>
It seems like there is only a problem with data relating to the building year

**FIX AGE ISSUE**

In [128]:
#lets do in the future median of district
value_count = df_features['Building age'].value_counts()[:63]
median = df_features['Building age'].median()
print(f'most popular age: {value_count}')
print(f'median age: {median}')
subdistricts = df_features.subdistrict.unique()
median = {}
avg = {}
for sub in subdistricts:
    sub_df = df_features[df_features['subdistrict'] == sub]
    median[sub] = sub_df['Building age'].median()
    avg[sub] = sub_df['Building age'].mean()
#median seems fine
for index, row in df_features.iterrows():
    age = (df_features.loc[index])
    if np.isnan(age['Building age']):
        key = age['subdistrict']
        med_val = median[key]
        #print(med_val)
        df_features.loc[index, 'Building age'] = median[key]
        #print(df_features.loc[index]['age'])

most popular age: 123.0    1033
53.0      397
58.0      378
48.0      311
63.0      303
Name: Building age, dtype: int64
median age: 58.0


**CHECKING NON ZERO VALUES**

In [129]:
for column in df_features.columns:
    print(f'Count Values that are not Zero for Column {column} : {(df_features[column] !=0).sum()} out {len(df_features)}')

Count Values that are not Zero for Column Building surface m² : 16299 out 16301
Count Values that are not Zero for Column Flat : 13987 out 16301
Count Values that are not Zero for Column Penthouse : 1190 out 16301
Count Values that are not Zero for Column Duplex : 539 out 16301
Count Values that are not Zero for Column House : 582 out 16301
Count Values that are not Zero for Column 1st Floor : 3301 out 16301
Count Values that are not Zero for Column 2nd Floor : 2500 out 16301
Count Values that are not Zero for Column 3th Floor : 1917 out 16301
Count Values that are not Zero for Column 4th Floor : 1539 out 16301
Count Values that are not Zero for Column 5th Floor : 1216 out 16301
Count Values that are not Zero for Column 6th Floor : 635 out 16301
Count Values that are not Zero for Column 7th Floor : 358 out 16301
Count Values that are not Zero for Column 8th Floor : 249 out 16301
Count Values that are not Zero for Column 9th Floor : 113 out 16301
Count Values that are not Zero for Colum

**feedback** <br><br>
<br> <br>
- Houses are almost always exterior, thereby deleted. This Variable is also not used in other Research.
<br> <br>
- First Findings Show That Almost 25% of the Observations Are Missing Energy Labels

**PRICE TREATMENT**

In [130]:
df_features['Log price'] = np.log(df_features['price'])

**FEATURES FOR MODEL**

for col in df_features.columns:
    print(col)

In [132]:
model_features = ['Building surface m²', "Flat","Penthouse","Duplex","House",
                  '1th Floor', '2nd Floor', '3th Floor', '4th Floor', '5th Floor', '6th Floor or higher' #, '7st', '8st', '9st',
       #'10st','11st_or_higher','6st_or_higher',
                  'Ground floor' #'Basement',  'Ground floor', 
                  'Elevator',
       'Swimming pool', 'Green Area', 'Garden', 'bedrooms', 'bathrooms', 'Building age','Air conditioning'
       'Terrace', 'Balcony', 'Heating', 'Parking Included'
                ,'Energy label consumption A','Energy label consumption B',
                'Energy_label_consumption C', 'Energy label consumption D',
                 'Energy_label_consumption E','Energy label consumption F','Energy label consumption G' #'Energy label consumption NA',
                  'New housing development',  #reference category
                  'Second hand/good condition','Second hand/needs renovating',
                 'District Eixample',
                  'District Ciutat Vella',
                   'District Sant Martí', 'District Sants-Montjuïc',
                   'District Horta Guinardó', 'District Gràcia', 'District Les Corts',
                   'District Nou Barris', 'District Sarrià-Sant Gervasi',
                   'District Sant Andreu']
target = ['Log price']

In [133]:
df_features

,Building surface m²,Flat,Penthouse,Duplex,House,1st Floor,2nd Floor,3th Floor,4th Floor,5th Floor,...,District Gràcia,District Les Corts,District Nou Barris,District Sarrià-Sant Gervasi,District Sant Andreu,subdistrict,price,check,6th Floor or higher,Log price
id,,,,,,,,,,,,,,,,,,,,,
31702050,50,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,Subdistrict La Dreta de l'Eixample,250000,2,1,12.429216
33148486,85,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,Subdistrict La Dreta de l'Eixample,770000,2,0,13.554146
39158016,318,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,Subdistrict La Dreta de l'Eixample,2500000,2,0,14.731801
40351755,190,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,Subdistrict La Dreta de l'Eixample,1350000,2,0,14.115615
93970881,100,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,Subdistrict La Dreta de l'Eixample,889000,2,0,13.697853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101135260,86,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,Subdistrict Sant Andreu,269000,2,0,12.502467
101135697,103,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,Subdistrict Baró de Viver,190000,2,0,12.154779
101135655,65,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,Subdistrict El Bon Pastor,125000,2,0,11.736069


In [134]:
print(len(df_features))
df_features = df_features.dropna(axis=0)
print(len(df_features))

16301
16297


In [135]:
print(f'Length DataFrame Before Dropping Observations with nu floor data: {len(df_features)}.')
df_features['no floor'] = 1- df_features[['1st Floor', '2nd Floor', '3th Floor', '4th Floor', '5th Floor', '6th Floor or higher', 'Ground Floor','House', 'Mezzanine']].sum(axis=1)
drop = df_features.drop(df_features[df_features['no floor'] != 1].index)
drop.to_excel('nofloordropped.xlsx')
df_features = df_features.drop(df_features[df_features['no floor'] == 1].index)
length = len(df_features[df_features['no floor']==1])
print(f'Length DataFrame After Dropping Observations with nu floor data: {len(df_features)}.')

Length DataFrame Before Dropping Observations with nu floor data: 16297.


C:\Users\niels\AppData\Local\Temp\ipykernel_18372\3959229495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['no floor'] = 1- df_features[['1st Floor', '2nd Floor', '3th Floor', '4th Floor', '5th Floor', '6th Floor or higher', 'Ground Floor','House', 'Mezzanine']].sum(axis=1)


Length DataFrame After Dropping Observations with nu floor data: 14899.


In [136]:
df_features.columns

Index(['Building surface m²', 'Flat', 'Penthouse', 'Duplex', 'House',
       '1st Floor', '2nd Floor', '3th Floor', '4th Floor', '5th Floor',
       '6th Floor', '7th Floor', '8th Floor', '9th Floor', '10th Floor',
       '11th Floor', '12th Floor', '13th Floor', '14th Floor', '15th Floor',
       '16th Floor', '17th Floor', '18th Floor', '19th Floor', '20th Floor',
       '21th Floor', '22th Floor', '23th Floor', '24th Floor', '25th Floor',
       '26th Floor', '27th Floor', '28th Floor', '29th Floor', '30th Floor',
       '31th Floor', '32th Floor', '33th Floor', '34th Floor', '35th Floor',
       '36th Floor', '37th Floor', '38th Floor', '39th Floor', 'Basement',
       'Ground Floor', 'Mezzanine', 'Semi-basement', 'Elevator',
       'Swimming pool', 'Green Area', 'Garden', 'Air conditioning', 'bedrooms',
       'bathrooms', 'Building age', 'Terrace', 'Balcony', 'Heating',
       'Parking included', 'New housing development',
       'Second hand/good condition', 'Second hand/needs r

**OUTDOOR FACILITIES**

In [137]:
df_features['Outdoor facilities'] = df_features['Swimming pool'] + df_features['Green Area'] + df_features['Garden']
df_features['Outdoor facilities'] = df_features['Swimming pool'] + df_features['Green Area'] + df_features['Garden']

## EXPORT 

In [138]:
export_housing_specific_variables = False
if export_housing_specific_variables:
    pd.to_excel("housing_specific_variables.xlsx")